In [26]:
import os,sys
import pandas as pd
import numpy as np
pd.set_option('display.max_rows',9999)

import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor

from sklearn import model_selection, preprocessing, feature_selection, ensemble, linear_model, metrics, decomposition

%matplotlib inline

In [50]:
train_data = pd.read_csv("train.csv")
#train_data

#converting all N.S. , No Sales and Nan values to 0

train_m = train_data.replace(regex=r'^N(.*?)$', value='0').fillna(0)

train_m['WeekEnding_Date']= pd.to_datetime(train_m['WeekEnding_Date'])

train_m['day'] = train_m['WeekEnding_Date'].dt.day
train_m['month'] = train_m['WeekEnding_Date'].dt.month
train_m['year'] = train_m['WeekEnding_Date'].dt.year
train_m = train_m.drop('WeekEnding_Date', axis=1)

new_cols = [col for col in train_m.columns if col != 'Average'] + ['Average']
train_m = train_m[new_cols]

train_m

,Kolkata_Average_Price,Kolkata_Ref_Price,Bangalore_Average_Price,Bangalore_Ref_Price,Cochin_Average_Price,Cochin_Ref_Price,Darjeeling_Average_Price,Darjeeling_Ref_Price,Ernakulam_Average_Price,Ernakulam_Ref_Price,Siliguri_Average_Price,Siliguri_Ref_Price,Guwahati_Average_Price,Guwahati_Ref_Price,day,month,year,Average
0,99.01,79.79,0,76.19,84.02,70.07,81.66,57.83,68.94,51.67,70.74,53.88,65.55,46.75,1,3,2009,69.700000
1,97.74,78.73,87.48,73.97,82.72,68.17,83.31,58.02,67.24,52.23,70.47,53.39,67.39,46.84,1,10,2009,70.550000
2,95.95,71.01,87.66,71.01,80.58,67.16,82.25,57.49,69.64,52.48,71.66,53.18,69.51,48.04,17,1,2009,69.830000
3,94.14,73.38,85.69,65.66,0,65.57,80.87,54.59,0,53.43,71.12,52.07,69.14,48.5,24,1,2009,67.846667
4,91.45,70.39,0,64.99,79.27,62.09,80.76,57.06,69.65,53.38,72.3,52.5,69.39,50.33,31,1,2009,67.196923
5,88.99,70.4,83.24,63.28,0,59.63,83.76,57.32,67.5,52.28,72.22,52.6,67.4,50.02,2,7,2009,66.818462
6,87.95,67.81,79.98,62.5,78.25,61.27,83.92,57.66,69.93,51.8,70.9,53.47,65.14,49.76,14,2,2009,67.167143
7,84.39,65.69,75.27,59.39,78.25,61.27,75.1,58.15,82.9,58.53,69.85,52.69,66.39,50,21,2,2009,66.990714
8,79.92,0,74.41,55.65,71.29,57.02,82.62,58.14,69.47,53.99,71.27,53.8,68.95,51,28,2,2009,65.194615
9,79.54,66.06,71.1,0,71.73,60.76,82.81,58.58,82.05,56,73.1,55.53,69.06,52.42,3,7,2009,67.595385


In [62]:
test_data = pd.read_csv("test.csv").fillna(0)
test_data_m = test_data.iloc[:,0:len(test_data.columns)-1]
test_data_m = test_data_m.replace(regex=r'^N(.*?)$', value='0')

test_data_m['WeekEnding_Date']= pd.to_datetime(test_data_m['WeekEnding_Date'])

test_data_m['day'] = test_data_m['WeekEnding_Date'].dt.day
test_data_m['month'] = test_data_m['WeekEnding_Date'].dt.month
test_data_m['year'] = test_data_m['WeekEnding_Date'].dt.year
test_data_m = test_data_m.drop('WeekEnding_Date', axis=1)


test_data_m

,Kolkata_Average_Price,Kolkata_Ref_Price,Bangalore_Average_Price,Bangalore_Ref_Price,Cochin_Average_Price,Cochin_Ref_Price,Darjeeling_Average_Price,Darjeeling_Ref_Price,Ernakulam_Average_Price,Ernakulam_Ref_Price,Siliguri_Average_Price,Siliguri_Ref_Price,Guwahati_Average_Price,Guwahati_Ref_Price,day,month,year
0,0.00,165.87,154.080,160.82,0.00,0.00,116.45,112.82,91.82,0.00,102.30,0,83.07,65.34,15,6,2019
1,171.35,173.12,0.000,0.00,0.00,0.00,0.00,0.00,90.68,76.34,0.00,0,81.34,0.00,22,6,2019
2,0.00,0.00,156.140,0.00,0.00,150.39,0.00,0.00,0.00,0.00,96.84,0,0.00,0.00,29,6,2019
3,0.00,0.00,158.040,0.00,0.00,146.68,111.24,111.47,0.00,0.00,97.87,0,0.00,0.00,7,6,2019
4,0.00,0.00,157.550,163.85,0.00,144.87,0.00,111.44,0.00,0.00,0.00,0,0.00,0.00,13,7,2019
5,179.47,0.00,0.000,0.00,0.00,0.00,107.52,111.32,77.77,80.61,89.80,0,0.00,0.00,20,7,2019
6,0.00,0.00,163.940,0.00,0.00,0.00,0.00,0.00,0.00,81.98,0.00,0,0.00,0.00,27,7,2019
7,182.14,0.00,0.000,0.00,0.00,142.63,0.00,0.00,0.00,83.83,0.00,0,0.00,0.00,8,3,2019
8,183.59,0.00,0.000,160.61,138.75,139.98,0.00,0.00,79.64,0.00,0.00,0,0.00,0.00,8,10,2019
9,0.00,169.98,0.000,0.00,0.00,0.00,0.00,121.85,0.00,0.00,0.00,98.66,0.00,0.00,17,8,2019


In [59]:
X= train_m.drop(train_m.columns[-1], axis=1).values
Y = train_m[train_m.columns[-1]].values


X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state=42)
base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(X_train, Y_train)
predictions = base_model.predict(X_test)
predictions
errors = abs(predictions-Y_test)

mape = 100 * np.mean(errors/Y_test)
accuracy = 100-mape
print(accuracy)

rmse = np.sqrt(metrics.mean_squared_error(Y_test,predictions))
mae = metrics.mean_absolute_error(Y_test,predictions)
r2 = round(metrics.r2_score(Y_test,predictions),2)
print(rmse,mae,r2)

97.99647310573008
3.5193688707716966 1.960075393894498 0.94


In [69]:
test_pred = base_model.predict(test_data_m)
test_pred_df = pd.DataFrame(test_pred, columns=['Average'])
test_pred_df.to_csv("submission2.csv", index=False)

test_pred_df

,Average
0,107.174550
1,95.700495
2,103.038769
3,105.785404
4,114.637694
5,102.592434
6,95.321953
7,100.115747
8,115.793714
9,95.006166


In [77]:
from statsmodels.tsa.arima_model import ARIMA




In [76]:
train_data = pd.read_csv("train.csv")
#train_data

#converting all N.S. , No Sales and Nan values to 0

train_m = train_data.replace(regex=r'^N(.*?)$', value='0').fillna(0)

train_m['WeekEnding_Date']= pd.to_datetime(train_m['WeekEnding_Date'])

train_m['day'] = train_m['WeekEnding_Date'].dt.day
train_m['month'] = train_m['WeekEnding_Date'].dt.month
train_m['year'] = train_m['WeekEnding_Date'].dt.year
#train_m = train_m.drop('WeekEnding_Date', axis=1)

new_cols = [col for col in train_m.columns if col != 'Average'] + ['Average']
train_m = train_m[new_cols]

train_m

,WeekEnding_Date,Kolkata_Average_Price,Kolkata_Ref_Price,Bangalore_Average_Price,Bangalore_Ref_Price,Cochin_Average_Price,Cochin_Ref_Price,Darjeeling_Average_Price,Darjeeling_Ref_Price,Ernakulam_Average_Price,Ernakulam_Ref_Price,Siliguri_Average_Price,Siliguri_Ref_Price,Guwahati_Average_Price,Guwahati_Ref_Price,day,month,year,Average
0,2009-03-01,99.01,79.79,0,76.19,84.02,70.07,81.66,57.83,68.94,51.67,70.74,53.88,65.55,46.75,1,3,2009,69.700000
1,2009-10-01,97.74,78.73,87.48,73.97,82.72,68.17,83.31,58.02,67.24,52.23,70.47,53.39,67.39,46.84,1,10,2009,70.550000
2,2009-01-17,95.95,71.01,87.66,71.01,80.58,67.16,82.25,57.49,69.64,52.48,71.66,53.18,69.51,48.04,17,1,2009,69.830000
3,2009-01-24,94.14,73.38,85.69,65.66,0,65.57,80.87,54.59,0,53.43,71.12,52.07,69.14,48.5,24,1,2009,67.846667
4,2009-01-31,91.45,70.39,0,64.99,79.27,62.09,80.76,57.06,69.65,53.38,72.3,52.5,69.39,50.33,31,1,2009,67.196923
5,2009-07-02,88.99,70.4,83.24,63.28,0,59.63,83.76,57.32,67.5,52.28,72.22,52.6,67.4,50.02,2,7,2009,66.818462
6,2009-02-14,87.95,67.81,79.98,62.5,78.25,61.27,83.92,57.66,69.93,51.8,70.9,53.47,65.14,49.76,14,2,2009,67.167143
7,2009-02-21,84.39,65.69,75.27,59.39,78.25,61.27,75.1,58.15,82.9,58.53,69.85,52.69,66.39,50,21,2,2009,66.990714
8,2009-02-28,79.92,0,74.41,55.65,71.29,57.02,82.62,58.14,69.47,53.99,71.27,53.8,68.95,51,28,2,2009,65.194615
9,2009-07-03,79.54,66.06,71.1,0,71.73,60.76,82.81,58.58,82.05,56,73.1,55.53,69.06,52.42,3,7,2009,67.595385


In [80]:
from sklearn.preprocessing import MinMaxScaler
train_m2 = train_m.drop(['sales','date'],axis=1)

X= train_m.drop(train_m.columns[-1], axis=1).values
Y = train_m[train_m.columns[-1]].values


X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state=42)
base_model = ARIMA(X_train, order=(1,1,1))
model_fit = base_model.fit()
predictions = model_fit.predict(X_test)
predictions
errors = abs(predictions-Y_test)

mape = 100 * np.mean(errors/Y_test)
accuracy = 100-mape
print(accuracy)

rmse = np.sqrt(metrics.mean_squared_error(Y_test,predictions))
mae = metrics.mean_absolute_error(Y_test,predictions)
r2 = round(metrics.r2_score(Y_test,predictions),2)
print(rmse,mae,r2)

ValueError: endog must be 1-d or 2-d with 1 column